In [1]:
import numpy as np
import torch


In [2]:
# # Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors using torch.from_numpy
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


#Linear regression model from scratch


In [5]:

# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-1.5231,  2.5235,  0.3256],
        [ 0.1646,  1.9012,  0.5161]], requires_grad=True)
tensor([ 0.3441, -0.4291], requires_grad=True)


In [7]:
# Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation).
def model(x):
    return x @ w.t() + b

In [8]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 72.2377, 161.1596],
        [104.6544, 214.8856],
        [224.8752, 298.5861],
        [-34.4496, 117.2078],
        [160.3039, 229.5703]], grad_fn=<AddBackward0>)


In [11]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 72.2377, 161.1596],
        [104.6544, 214.8856],
        [224.8752, 298.5861],
        [-34.4496, 117.2078],
        [160.3039, 229.5703]], grad_fn=<AddBackward0>)


In [9]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Loss function
Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

Calculate the difference between the two matrices (preds and targets).
Square all elements of the difference matrix to remove negative values.
Calculate the average of the elements in the resulting matrix.
The result is a single number, known as the mean squared error (MSE).

In [12]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [13]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(8586.0879, grad_fn=<DivBackward0>)


In [14]:
#gradients
# Compute gradients
loss.backward()

In [15]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-1.5231,  2.5235,  0.3256],
        [ 0.1646,  1.9012,  0.5161]], requires_grad=True)
tensor([[ 2149.0308,  4086.1260,  2055.1006],
        [ 9446.9551, 10476.3691,  6304.0283]])


In [16]:

w
w.grad



tensor([[ 2149.0308,  4086.1260,  2055.1006],
        [ 9446.9551, 10476.3691,  6304.0283]])

In [17]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [18]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(8586.0879, grad_fn=<DivBackward0>)


In [19]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [20]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 67.0472, 144.5323],
        [ 97.7875, 193.0339],
        [216.3378, 272.6714],
        [-39.1594, 100.7335],
        [153.4595, 208.5806]], grad_fn=<AddBackward0>)


In [21]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(6178.5205, grad_fn=<DivBackward0>)


In [22]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[1609.1897, 3494.9980, 1692.8585],
        [7729.8477, 8629.7803, 5164.9575]])
tensor([22.8945, 91.9104])


In [23]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [24]:
print(w)
print(b)

tensor([[-1.5607,  2.4477,  0.2881],
        [-0.0072,  1.7101,  0.4014]], requires_grad=True)
tensor([ 0.3435, -0.4312], requires_grad=True)


In [25]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(4551.3389, grad_fn=<DivBackward0>)


In [26]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [27]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(373.3452, grad_fn=<DivBackward0>)


In [28]:
# Predictions
preds

tensor([[ 49.8555,  69.4545],
        [ 71.8495,  94.2994],
        [154.0015, 148.8201],
        [-22.2857,  31.8872],
        [109.2776, 110.9616]], grad_fn=<AddBackward0>)

In [29]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

LINEAR REGRESSION USING PYTORCH BUILTS-INS


In [30]:
import torch.nn as nn  #torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [31]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [33]:
#a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.
from torch.utils.data import TensorDataset

In [34]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [35]:
from torch.utils.data import DataLoader

In [36]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [37]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [101.,  44.,  37.],
        [ 87., 135.,  57.],
        [ 88., 134.,  59.],
        [102.,  43.,  37.]])
tensor([[ 81., 101.],
        [ 21.,  38.],
        [118., 134.],
        [118., 132.],
        [ 22.,  37.]])


In [38]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1311,  0.0081,  0.0997],
        [-0.2596, -0.4998, -0.1769]], requires_grad=True)
Parameter containing:
tensor([ 0.1229, -0.0732], requires_grad=True)


In [39]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.1311,  0.0081,  0.0997],
         [-0.2596, -0.4998, -0.1769]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1229, -0.0732], requires_grad=True)]

In [40]:
# Generate predictions
preds = model(inputs)
preds

tensor([[  -4.6252,  -60.1146],
        [  -4.7237,  -78.9976],
        [  -4.4266,  -99.8871],
        [  -9.2194,  -54.5864],
        [  -1.1764,  -78.3465],
        [  -4.7644,  -59.8744],
        [  -4.6321,  -78.6747],
        [  -4.4580, -100.3236],
        [  -9.0802,  -54.8266],
        [  -0.9456,  -78.2639],
        [  -4.5336,  -59.7918],
        [  -4.8629,  -78.7574],
        [  -4.5182, -100.2099],
        [  -9.4502,  -54.6690],
        [  -1.0372,  -78.5867]], grad_fn=<AddmmBackward0>)

In [41]:
# Import nn.functional
import torch.nn.functional as F
# Define loss function
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

tensor(18882.8340, grad_fn=<MseLossBackward0>)


In [42]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

    # Repeat for given number of epochs
    for epoch in range(num_epochs):

        # Train with batches of data
        for xb,yb in train_dl:

            # 1. Generate predictions
            pred = model(xb)

            # 2. Calculate loss
            loss = loss_fn(pred, yb)

            # 3. Compute gradients
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset the gradients to zero
            opt.zero_grad()

        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [43]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 289.4660
Epoch [20/100], Loss: 202.5654
Epoch [30/100], Loss: 62.4408
Epoch [40/100], Loss: 115.4308
Epoch [50/100], Loss: 35.3669
Epoch [60/100], Loss: 79.9300
Epoch [70/100], Loss: 49.7228
Epoch [80/100], Loss: 27.8382
Epoch [90/100], Loss: 22.9303
Epoch [100/100], Loss: 22.2503


In [44]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.9689,  71.6674],
        [ 81.0956,  98.7083],
        [118.2595, 134.5000],
        [ 26.7912,  45.2705],
        [ 96.8003, 110.9474],
        [ 56.8485,  70.7238],
        [ 80.7351,  98.4071],
        [118.4740, 134.9336],
        [ 27.9117,  46.2141],
        [ 97.5603, 111.5898],
        [ 57.6084,  71.3662],
        [ 79.9751,  97.7647],
        [118.6200, 134.8012],
        [ 26.0313,  44.6280],
        [ 97.9208, 111.8910]], grad_fn=<AddmmBackward0>)

In [45]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [47]:
!git config --global user.name "Nabila84"
!git config --global user.email "khan.nik841@gmail.com"


In [46]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.5198, 68.6398]], grad_fn=<AddmmBackward0>)

In [48]:
!git clone <https://github.com/Nabila84/ML-projects>


/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `git clone <https://github.com/Nabila84/ML-projects>'
